In [3]:
import numpy as np
import matplotlib.pyplot as plt
import requests
# import pypsa
import json
import pandas as pd
import math
from tabulate import tabulate
import textwrap

from IPython.display import display



## Setup
The Following code is the description of RD1

In [8]:
class build:
    def __init__(self, hexbus_modules, interconnects, hexframe_structure, tfrp_module, spg_module, wpt_module, propulsion_attitude_control, mppr_arms, initial_propellant_load):
        self.hexbus_modules = hexbus_modules
        self.interconnects = interconnects
        self.hexframe_structure = hexframe_structure
        self.tfrp_module = tfrp_module
        self.spg_module = spg_module
        self.wpt_module = wpt_module
        self.propulsion_attitude_control = propulsion_attitude_control
        self.mppr_arms = mppr_arms
        self.initial_propellant_load = initial_propellant_load


class buildmass:
    def __init__(self, hexbus_modules_m, interconnects_m, hexframe_structure_m, tfrp_module_m, spg_module_m, wpt_module_m, propulsion_attitude_control_m, mppr_arms_m, initial_propellant_load_m, totalmass):
        self.hexbus_modules_m = hexbus_modules_m
        self.interconnects_m = interconnects_m
        self.hexframe_structure_m = hexframe_structure_m
        self.tfrp_module_m = tfrp_module_m
        self.spg_module_m = spg_module_m
        self.wpt_module_m = wpt_module_m
        self.propulsion_attitude_control_m = propulsion_attitude_control_m
        self.mppr_arms_m = mppr_arms_m
        self.initial_propellant_load_m = initial_propellant_load_m
        self.totalmass = totalmass

In [9]:
drm1 = build(280, 1674, 159, 29, 223, 217, 6, 41, 6)
drm1_mass = buildmass(13.9, 1, 13.6, 4, 5.1, 12, 10, 10, 10, 0)
drm2 = build(445, 2658, 214, 35, 337, 331, 6, 57, 6)
drm2_mass = buildmass(13.9, 1, 7, 2, 5.1, 12, 10, 10, 10, 0)
drm3_c1 = build(2365, 14178, 214, 35, 2319, 2269, 50, 76, 50)   #c1 refers to the power outage, in this case = 2MW
drm3_c1_mass = buildmass(24, 1, 7, 2, 21, 47, 16, 10, 58, 0)
drm3_c2= build(10301, 61782, 552, 113, 10019, 9919, 100, 237, 100)    #c2 refers to the power outage, in this case = 18MW
drm3_c2_mass = buildmass(24, 1, 55, 80, 21, 47, 36, 10, 130, 0)
drm4 = build(131808, 790722, 8360, 1750, 128127, 127927, 200, 2078, 200)
drm4_mass = buildmass(24, 1, 54, 79, 8, 47, 195, 10, 718, 0)
drm5 = build(392341, 2353662, 18444, 4305, 383619, 383419, 200, 4888, 200) #Delivers 2GW
drm5_mass = buildmass(24, 1, 54, 79, 8, 47, 578, 10, 2128, 0)

drm = [drm1, drm2, drm3_c1, drm3_c2, drm4, drm5]
drm_mass = [drm1_mass, drm2_mass, drm3_c1_mass, drm3_c2_mass, drm4_mass, drm5_mass]

drm_totalmass = np.zeros(6)


for i in range(len(drm)):
    masses = []
    masshex = drm[i].hexbus_modules * drm_mass[i].hexbus_modules_m
    massinterconnects = drm[i].interconnects * drm_mass[i].interconnects_m
    masshexframe = drm[i].hexframe_structure * drm_mass[i].hexframe_structure_m
    masstfrp = drm[i].tfrp_module * drm_mass[i].tfrp_module_m
    massspg = drm[i].spg_module * drm_mass[i].spg_module_m
    masswpt = drm[i].wpt_module * drm_mass[i].wpt_module_m
    masspropultion = drm[i].propulsion_attitude_control * drm_mass[i].propulsion_attitude_control_m
    massmppr = drm[i].mppr_arms * drm_mass[i].mppr_arms_m
    massinpropultion = drm[i].initial_propellant_load * drm_mass[i].initial_propellant_load_m

    results = np.append(masses, [masshex, massinterconnects, masshexframe, masstfrp, massspg, masswpt, masspropultion, massmppr, massinpropultion])

    drm_totalmass[i] = np.sum(results)



for k in range(len(drm_mass)):
    drm_mass[k].totalmass = drm_totalmass[k]

### RD1 & RD2 Energy Generation
The different energy generations for RD1 and RD2

In [10]:
drm1_power_delivery = 0.030 #in MW -- 30kW
drm2_power_delivery = 0.2 #in MW -- 200KW
drm3_c1_power_delivery = 2 #in MW
drm3_c2_power_delivery = 18 #in MW
drm4_power_delivery = 500 #in MW
drm5_power_delivery = 2000 #in MW

In [11]:
rd2_subpanel = 9500
rd2_power = 420 #420 W
DC_efficiency = 0.35
RF_efficiency = 0.85
rd2_power_transmission = rd2_power * DC_efficiency * RF_efficiency

1 TWH = 1TW * hours in a year
twh / years hours = TW

TW = 1e6 MW



### Get data
The following block of code is the API used to get the data for the yearly energy generation and consumption of a country.

In [12]:
def get_country_generation(country, year):
    my_api_key = "f5172a17-bcda-49e4-afa5-7cafa8613042"
    base_url = "https://api.ember-climate.org"
    query_url_generation = (
        f"{base_url}/v1/electricity-generation/yearly"
        f"?entity_code={country}&is_aggregate_series=false&start_date={year}&api_key={my_api_key}"
    )

    response_generation = requests.get(query_url_generation)

    if response_generation.status_code == 200:
        data_generation = response_generation.json()
        df_generation = pd.json_normalize(data_generation['data'])
        return pd.DataFrame(df_generation)
    else:
        print(f"Error: {response_generation.status_code}")
        return pd.DataFrame()  # Return an empty DataFrame in case of an error

In [13]:
## NOT USED AT THE MOMENT - TO DO AFTER GENERATION
def get_country_consumption(country, year):
    my_api_key = "f5172a17-bcda-49e4-afa5-7cafa8613042"
    base_url = "https://api.ember-climate.org"
    query_url_consumption = (f"{base_url}/v1/electricity-demand/yearly"+ f"?entity_code={country}&is_aggregate_series=false&start_date={year}&api_key={my_api_key}")

    response_consumption = requests.get(query_url_consumption)

    if response_consumption.status_code == 200:
        data_consumption = response_consumption.json()
        df_consumption = pd.json_normalize(data_consumption['data'])
        return pd.DataFrame(df_consumption)
    
    else:
        print(f"Error: {response_consumption.status_code}")
        return pd.DataFrame()  # Return an empty DataFrame in case of an error

In [14]:
# Function to get and validate country code
def get_country_code():
    country = input("Which country would you like to use? ")
    while len(country) != 3:
        country = input("Please use the three-letter code for the country: ")
    return country.upper()

## Generation Data

In [18]:
# Get inputs from the user
country = get_country_code()
year = input("Which year? ")

# Get the data 
df_country_gen = get_country_generation(country, year)



# Print or further process the dataframe
df_country_gen



,entity,entity_code,is_aggregate_entity,date,series,is_aggregate_series,generation_twh,share_of_generation_pct
0,Brazil,BRA,False,2020,Bioenergy,False,58.74,9.34
1,Brazil,BRA,False,2020,Coal,False,17.75,2.82
2,Brazil,BRA,False,2020,Gas,False,59.48,9.46
3,Brazil,BRA,False,2020,Hydro,False,396.38,63.04
4,Brazil,BRA,False,2020,Net imports,False,24.72,3.93
5,Brazil,BRA,False,2020,Nuclear,False,14.05,2.23
6,Brazil,BRA,False,2020,Other fossil,False,14.56,2.32
7,Brazil,BRA,False,2020,Solar,False,10.75,1.71
8,Brazil,BRA,False,2020,Wind,False,57.05,9.07
9,Brazil,BRA,False,2021,Bioenergy,False,55.72,8.49


In [19]:
# retain useful cols and energy sources of interest
df_country_gen_clean = df_country_gen[['entity','date','series','generation_twh']]
df_country_gen_clean = df_country_gen_clean[df_country_gen_clean['series'].isin(['Bioenergy', 'Coal', 'Gas', 'Other fossil'])]
df_country_gen_clean['generation_twh'] = df_country_gen_clean['generation_twh'] * 1e6 / (24 * 365)
df_country_gen_clean = df_country_gen_clean.rename(columns={'entity': 'country', 'date': 'year', 'series': 'source', 'generation_twh': 'energy_mw'})
df_country_gen_clean

,country,year,source,energy_mw
0,Brazil,2020,Bioenergy,6705.479452
1,Brazil,2020,Coal,2026.255708
2,Brazil,2020,Gas,6789.954338
6,Brazil,2020,Other fossil,1662.100457
9,Brazil,2021,Bioenergy,6360.730594
10,Brazil,2021,Coal,2767.123288
11,Brazil,2021,Gas,9926.940639
15,Brazil,2021,Other fossil,2583.333333
18,Brazil,2022,Bioenergy,6281.963470
19,Brazil,2022,Coal,1621.004566


In [20]:
#Get the data
df_country_prod = get_country_consumption(country, year)

#Print or further process the dataframe
df_country_prod.columns


Index(['entity', 'entity_code', 'is_aggregate_entity', 'date', 'demand_twh',
       'demand_mwh_per_capita'],
      dtype='object')

In [21]:
# Retain useful cols 
df_country_prod_clean = df_country_prod[['entity', 'date', 'demand_twh']]
df_country_prod_clean['demand_twh'] = df_country_prod_clean['demand_twh'] *1e6 / (24 * 365)
#df_country_prod_clean['demand_mwh_per_capita'] = df_country_prod_clean['demand_mwh_per_capita'] * 1 / (24 * 365)
df_country_prod_clean = df_country_prod_clean.rename(columns={'entity': 'country', 'date': 'year', 'demand_twh': 'demand_mw', 'demand_mwh_per_capita': 'demand_mw_per_capita'})
df_country_prod_clean

/var/folders/0y/496_zl5905jfg059dd524bvm0000gn/T/ipykernel_2270/378527193.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_country_prod_clean['demand_twh'] = df_country_prod_clean['demand_twh'] *1e6 / (24 * 365)


,country,year,demand_mw
0,Brazil,2020,74598.173516
1,Brazil,2021,77535.388128
2,Brazil,2022,78776.255708
3,Brazil,2023,82522.831050


In [22]:
# satellites and energy generation
satellite_generation = {
    'drm1': 0.030,  # 30 kW
    'drm2': 0.2,    # 200 kW
    'drm3_c1': 2,   # 2 MW
    'drm3_c2': 18,  # 18 MW
    'drm4': 500,    # 500 MW
    'drm5': 2000,   # 2000 MW
    'rd2': 0.00042,     # 420 W = 0.42 MW
    'rd2_complete': 0.00042*9500
}

In [23]:
# Calculate the number of each type of satellite needed
def calculate_num_satetilles(energy_mw, satellite_generation):
  return {satellite: math.ceil(energy_mw / energy_gen) for satellite, energy_gen in satellite_generation.items()}

In [15]:
# # OLD CODE - converted to function in code block below

# # Create list of dictionaries [{source: 'first', sat1: '', sat2:'', ... satX: ''},...,{source: 'last', sat1: '', sat2:'', ... satX: ''}]
# data_dicts = []

# # Calc num of sats needed for each energy type (and build a list of dicts)
# for _, row in df_country_gen_clean.iterrows():
#     source = row['source']
#     energy_mw = row['energy_mw']

#     # Create dictionary with the source and calculated satellite numbers
#     ret_dict = {'source': source}
#     ret_dict.update(calculate_num_satetilles(energy_mw, satellite_generation))

#     # Append the dictionary to the list
#     data_dicts.append(ret_dict)

# data_dicts

In [24]:
# Create list of dictionaries [{source: 'first', sat1: '', sat2:'', ... satX: ''},...,{source: 'last', sat1: '', sat2:'', ... satX: ''}]

def calculate_sats_needed(df, satellite_generation):
    """Calculate the number of satellites needed for each energy source and return a list of dictionaries."""
    data_dicts = []

    # Iterate over each row in the DataFrame
    for _, row in df.iterrows():
        source = row['source']
        energy_mw = row['energy_mw']

        # Create dictionary with the source and calculated satellite numbers
        ret_dict = {'source': source}
        ret_dict.update(calculate_num_satetilles(energy_mw, satellite_generation))

        # Append the dictionary to the list
        data_dicts.append(ret_dict)

    return data_dicts


def calculate_sats_needed_prod(df, satellite_generation):
    
    data_dicts = []
    
    for _, row in df.iterrows():
        demand = row['demand_mw']
        
        ret_dict = {'demand_mw': demand}
        ret_dict.update(calculate_num_satetilles(demand, satellite_generation))
        
        data_dicts.append(ret_dict)
    return data_dicts

        

In [25]:
data_dictionaries = calculate_sats_needed(df_country_gen_clean, satellite_generation)

data_dictionaries

[{'source': 'Bioenergy',
  'drm1': 223516,
  'drm2': 33528,
  'drm3_c1': 3353,
  'drm3_c2': 373,
  'drm4': 14,
  'drm5': 4,
  'rd2': 15965428,
  'rd2_complete': 1681},
 {'source': 'Coal',
  'drm1': 67542,
  'drm2': 10132,
  'drm3_c1': 1014,
  'drm3_c2': 113,
  'drm4': 5,
  'drm5': 2,
  'rd2': 4824419,
  'rd2_complete': 508},
 {'source': 'Gas',
  'drm1': 226332,
  'drm2': 33950,
  'drm3_c1': 3395,
  'drm3_c2': 378,
  'drm4': 14,
  'drm5': 4,
  'rd2': 16166558,
  'rd2_complete': 1702},
 {'source': 'Other fossil',
  'drm1': 55404,
  'drm2': 8311,
  'drm3_c1': 832,
  'drm3_c2': 93,
  'drm4': 4,
  'drm5': 1,
  'rd2': 3957383,
  'rd2_complete': 417},
 {'source': 'Bioenergy',
  'drm1': 212025,
  'drm2': 31804,
  'drm3_c1': 3181,
  'drm3_c2': 354,
  'drm4': 13,
  'drm5': 4,
  'rd2': 15144597,
  'rd2_complete': 1595},
 {'source': 'Coal',
  'drm1': 92238,
  'drm2': 13836,
  'drm3_c1': 1384,
  'drm3_c2': 154,
  'drm4': 6,
  'drm5': 2,
  'rd2': 6588389,
  'rd2_complete': 694},
 {'source': 'Gas',
 

In [26]:
data_dictionaries_prod = calculate_sats_needed_prod(df_country_prod_clean, satellite_generation)

data_dictionaries_prod

[{'demand_mw': 74598.17351598173,
  'drm1': 2486606,
  'drm2': 372991,
  'drm3_c1': 37300,
  'drm3_c2': 4145,
  'drm4': 150,
  'drm5': 38,
  'rd2': 177614699,
  'rd2_complete': 18697},
 {'demand_mw': 77535.38812785388,
  'drm1': 2584513,
  'drm2': 387677,
  'drm3_c1': 38768,
  'drm3_c2': 4308,
  'drm4': 156,
  'drm5': 39,
  'rd2': 184608067,
  'rd2_complete': 19433},
 {'demand_mw': 78776.25570776255,
  'drm1': 2625876,
  'drm2': 393882,
  'drm3_c1': 39389,
  'drm3_c2': 4377,
  'drm4': 158,
  'drm5': 40,
  'rd2': 187562514,
  'rd2_complete': 19744},
 {'demand_mw': 82522.83105022831,
  'drm1': 2750762,
  'drm2': 412615,
  'drm3_c1': 41262,
  'drm3_c2': 4585,
  'drm4': 166,
  'drm5': 42,
  'rd2': 196482932,
  'rd2_complete': 20683}]

In [27]:
# Convert list of dictionaries to DataFrame
new_df_to_merge = pd.DataFrame(data_dictionaries)

# Merge the new DataFrame with the original DataFrame on the 'source' column
df_country_gen_final = pd.merge(df_country_gen_clean, new_df_to_merge, on='source', how='left')

In [28]:
df_country_gen_final

,country,year,source,energy_mw,drm1,drm2,drm3_c1,drm3_c2,drm4,drm5,rd2,rd2_complete
0,Brazil,2020,Bioenergy,6705.479452,223516,33528,3353,373,14,4,15965428,1681
1,Brazil,2020,Bioenergy,6705.479452,212025,31804,3181,354,13,4,15144597,1595
2,Brazil,2020,Bioenergy,6705.479452,209399,31410,3141,349,13,4,14957056,1575
3,Brazil,2020,Bioenergy,6705.479452,212443,31867,3187,355,13,4,15174495,1598
4,Brazil,2020,Coal,2026.255708,67542,10132,1014,113,5,2,4824419,508
...,...,...,...,...,...,...,...,...,...,...,...,...
59,Brazil,2023,Gas,4304.794521,143494,21524,2153,240,9,3,10249511,1079
60,Brazil,2023,Other fossil,1384.703196,55404,8311,832,93,4,1,3957383,417
61,Brazil,2023,Other fossil,1384.703196,86112,12917,1292,144,6,2,6150794,648
62,Brazil,2023,Other fossil,1384.703196,47185,7078,708,79,3,1,3370298,355


## Generation Data Function

In [29]:
def get_satellite_requirements_for_country_energy_generation(country, year, satellite_generation):

  # Get the data
  df_country_gen = get_country_generation(country, year)

  # Clean data
  df_country_gen_clean = df_country_gen[['entity','date','series','generation_twh']]
  df_country_gen_clean = df_country_gen_clean[df_country_gen_clean['series'].isin(['Bioenergy', 'Coal', 'Gas', 'Other fossil'])]
  df_country_gen_clean['generation_twh'] = df_country_gen_clean['generation_twh'] * 1e6 / (24 * 365)
  df_country_gen_clean = df_country_gen_clean.rename(columns={'entity': 'country', 'date': 'year', 'series': 'source', 'generation_twh': 'energy_mw'})

  data_dicts = calculate_sats_needed(df_country_gen_clean, satellite_generation)

  # Convert list of dictionaries to DataFrame
  new_df_to_merge = pd.DataFrame(data_dicts)

  # Merge the new DataFrame with the original DataFrame on the 'source' column
  df_country_gen_final = pd.merge(df_country_gen_clean, new_df_to_merge, on='source', how='left')

  return df_country_gen_final


In [31]:
def get_satellite_requirements_for_country_energy_production(country, year, satellite_generation):

  # Get the data
  df_country_prod = get_country_generation(country, year)

  # Clean data
  df_country_prod_clean = df_country_prod[['entity', 'date', 'demand_twh']]
  df_country_prod_clean['demand_twh'] = df_country_prod_clean['demand_twh'] *1e6 / (24 * 365)
  #df_country_prod_clean['demand_mwh_per_capita'] = df_country_prod_clean['demand_mwh_per_capita'] * 1 / (24 * 365)
  df_country_prod_clean = df_country_prod_clean.rename(columns={'entity': 'country', 'date': 'year', 'demand_twh': 'demand_mw'})


  data_dicts = calculate_sats_needed_prod(df_country_prod_clean, satellite_generation)

  # Convert list of dictionaries to DataFrame
  new_df_to_merge = pd.DataFrame(data_dicts)

  # Merge the new DataFrame with the original DataFrame on the 'source' column
  df_country_prod_final = pd.merge(df_country_prod_clean, new_df_to_merge, on='demand_mw', how='left')

  return df_country_prod_final


In [32]:
# Get inputs from the user
country = get_country_code()
year = input("Which year? (Only 2023 supported) ") # restrict range of API call for other years

# Constants
satellite_generation = {
    'drm1': 0.030,  # 30 kW
    'drm2': 0.2,    # 200 kW
    'drm3_c1': 2,   # 2 MW
    'drm3_c2': 18,  # 18 MW
    'drm4': 500,    # 500 MW
    'drm5': 2000,   # 2000 MW
    'rd2': 0.00042,     # 420 W = 0.42 MW
    'rd2_complete': 0.00042*9500
}

# Get the data
df_country_gen = get_satellite_requirements_for_country_energy_generation(country, year, satellite_generation)

# Print or further process the dataframe
df_country_gen

,country,year,source,energy_mw,drm1,drm2,drm3_c1,drm3_c2,drm4,drm5,rd2,rd2_complete
0,Brazil,2020,Bioenergy,6705.479452,223516,33528,3353,373,14,4,15965428,1681
1,Brazil,2020,Bioenergy,6705.479452,212025,31804,3181,354,13,4,15144597,1595
2,Brazil,2020,Bioenergy,6705.479452,209399,31410,3141,349,13,4,14957056,1575
3,Brazil,2020,Bioenergy,6705.479452,212443,31867,3187,355,13,4,15174495,1598
4,Brazil,2020,Coal,2026.255708,67542,10132,1014,113,5,2,4824419,508
...,...,...,...,...,...,...,...,...,...,...,...,...
59,Brazil,2023,Gas,4304.794521,143494,21524,2153,240,9,3,10249511,1079
60,Brazil,2023,Other fossil,1384.703196,55404,8311,832,93,4,1,3957383,417
61,Brazil,2023,Other fossil,1384.703196,86112,12917,1292,144,6,2,6150794,648
62,Brazil,2023,Other fossil,1384.703196,47185,7078,708,79,3,1,3370298,355


In [33]:
# Get inputs from the user
country = get_country_code()
year = input("Which year? (Only 2023 supported) ") # restrict range of API call for other years

# Constants
satellite_generation = {
    'drm1': 0.030,  # 30 kW
    'drm2': 0.2,    # 200 kW
    'drm3_c1': 2,   # 2 MW
    'drm3_c2': 18,  # 18 MW
    'drm4': 500,    # 500 MW
    'drm5': 2000,   # 2000 MW
    'rd2': 0.00042,     # 420 W = 0.42 MW
    'rd2_complete': 0.00042*9500
}

# Get the data
df_country_prod = get_satellite_requirements_for_country_energy_production(country, year, satellite_generation)

# Print or further process the dataframe
df_country_prod

KeyError: "['demand_twh'] not in index"

### Desaster Scenario

Essential Power:

- Hospitals
- Water Supply and Sewage
- Emergency Services i.e. Police...
- Communications
- Food Supply and Distribution
- Critical Infrastructurers

For around a city of 500 000 people:

need approx 215 - 370 MW









RF to DC via Rectenna conversion efficiency:

Approx e = 0.86 for 11 dBm and 5.2 - 5.9 GHz

and e = 0.9 for 2.45 GHz

### Design Table

In [15]:
Large_array_description = "Placing large satellites in GEO, which captures solar energy using solar cells, converts the energy into RF signals, rectennas capture the RF signals to turn into electircal signals"

Sun_tower_description = "Cyclindrical modules stacked together forming a tower in LEO, where the tower spins on its axis to stabilize maintaining its orientation towards Earth and thefdes Sun"

Modular_Symmetrical_concentrator_description = "Large modular array that concentrates solar light into small specific spot, often being a high efficiency solar cells or even heat engines in GEO. Energy is converted to RF and transmitted to Earth"

SPS_via_arbitrarily_large_pahsed_array_description = "Large array of small modules which captures and transmits energy from GEO"

Space_based_reflectors_description = "Solar Reflectors in GEO, LEO, L1 or L2, that reflect sunlight to a specific point on Earth. It does not directly generate power, it redirects sunlight to places in need, which can include solar farms"

Inflatable_thinfilm_structure_description = "Lightweight inflatable strunctures used to deploy large thin refelctors, solar arrays. Could be used for power generation, reflecting sunglight or even radiation sheilding"

Hyper_Modular_SBSP_Description = "Extremely modular design which allows for easy scalability and incremental expansion"

Terthered_description = "Modular satellites which can be connected via long tethers in GEO"

df = pd.DataFrame({
    "design": ["Large array of solar panels", "Sun tower", "Modular Symmetrical concentrator", "SPS via arbitrarily large phased array", "Space based reflectors", "Inflatable thin-film structures", "Hypermodular SBSP designs", "Tethered satellite systems"],
    "Discription": [Large_array_description, Sun_tower_description, Modular_Symmetrical_concentrator_description, SPS_via_arbitrarily_large_pahsed_array_description, Space_based_reflectors_description, Inflatable_thinfilm_structure_description, Hyper_Modular_SBSP_Description, Terthered_description],
    "Features": ["Scalable", "Scalable and in LEO", "Scalable and flexible deployment", "Highly Scalable and modular system ", "Highly scalable", "Multi-Purpose, easy deployment", "Highly Scalable, Modular, expansion by part", "Highly Scalable, reduces the need of large rigid structures"],
    "Size": ["1-5km", "1-2km height", "1-2km diameter", "1-10km", "meters to several km", "hundreds of meters", "km", "Several "],
    "Weight": ["10 000 - 100 000 tons", "5 000 - 20 000 tons", "10 000 - 30 000 tons", "10 000 - 100 000 tons", "10-50 tons", "A few tons", "10 000 - 100 000 tons", "10 000 - 50 000 tons"],
    #"Energy Consumption": ["", "", "", "", "", "", "", ""],
    "Energy Production": ["1-5 GW", "0.5-1GW", "1-5GW", "1-10GW", "Boosts solar farms by +10%", "MW - GW", "GW", "1-5 GW"],
    "Cost": ["20-500 billion", "10 - 100 billion", "20 - 150 Billion", "20 - 500 Billion", "0.5-1 billion", "10 - 100 million", "50 - 500 billion", "100 - 500 million"]
    
    
})




df

# Export to LaTeX
latex_code = df.to_latex(index=False, column_format="p{2.5cm} p{7cm} p{4.5cm} p{2.5cm} p{3.5cm} p{2.5cm} p{2.5cm} p{2.5cm}")

# Save to a .tex file
with open("table_output.tex", "w") as f:
    f.write(latex_table)


df



,design,Discription,Features,Size,Weight,Energy Production,Cost
0,Large array of solar panels,"Placing large satellites in GEO, which capture...",Scalable,1-5km,10 000 - 100 000 tons,1-5 GW,20-500 billion
1,Sun tower,Cyclindrical modules stacked together forming ...,Scalable and in LEO,1-2km height,5 000 - 20 000 tons,0.5-1GW,10 - 100 billion
2,Modular Symmetrical concentrator,Large modular array that concentrates solar li...,Scalable and flexible deployment,1-2km diameter,10 000 - 30 000 tons,1-5GW,20 - 150 Billion
3,SPS via arbitrarily large phased array,Large array of small modules which captures an...,Highly Scalable and modular system,1-10km,10 000 - 100 000 tons,1-10GW,20 - 500 Billion
4,Space based reflectors,"Solar Reflectors in GEO, LEO, L1 or L2, that r...",Highly scalable,meters to several km,10-50 tons,Boosts solar farms by +10%,0.5-1 billion
5,Inflatable thin-film structures,Lightweight inflatable strunctures used to dep...,"Multi-Purpose, easy deployment",hundreds of meters,A few tons,MW - GW,10 - 100 million
6,Hypermodular SBSP designs,Extremely modular design which allows for easy...,"Highly Scalable, Modular, expansion by part",km,10 000 - 100 000 tons,GW,50 - 500 billion
7,Tethered satellite systems,Modular satellites which can be connected via ...,"Highly Scalable, reduces the need of large rig...",Several,10 000 - 50 000 tons,1-5 GW,100 - 500 million


In [4]:
data = {
    'Aspect': [
        'Modularity',
        'Efficiency',
        'Lightweight',
        'Size',
        'Weight',
        'Energy Production',
        'Cost'
    ],
    'Large Array of Solar Panels': [
        '- Limited modularity\n- Monolithic structure',
        '- Good energy capture\n- Large surface area',
        '- Heavy\n- Large structural mass',
        '1–5 km',
        '10,000–100,000 tons',
        '1–5 GW',
        '$20–500 billion'
    ],
    'Sun Tower Design': [
        '- Moderate modularity\n- Add collectors along tower',
        '- High efficiency\n- Constant solar exposure',
        '- Heavy\n- Substantial structural components',
        '1–2 km height',
        '5,000–20,000 tons',
        '0.5–1 GW',
        '$10–100 billion'
    ],
    'Modular Symmetrical Concentrator': [
        '- High modularity\n- Interchangeable modules',
        '- High efficiency\n- Concentrates sunlight',
        '- Moderate weight\n- Mirrors/lenses reduce mass',
        '1–2 km diameter',
        '10,000–30,000 tons',
        '1–5 GW',
        '$20–150 billion'
    ],
    'Solar Power Satellite via Arbitrarily Large Phased Array': [
        '- High modularity\n- Expandable phased array',
        '- High efficiency\n- Precise energy transmission',
        '- Moderate to heavy\n- Large arrays increase mass',
        '1–10 km',
        '10,000–100,000 tons',
        '1–10 GW',
        '$20–500 billion'
    ],
    'Space-Based Reflectors': [
        '- Moderate modularity\n- Add reflectors as needed',
        '- Enhances efficiency\n- Redirects sunlight',
        '- Moderate weight\n- Lightweight materials possible',
        'Meters to several km',
        '10–50 tons',
        'Boosts solar farms by +10%',
        '$0.5–1 billion'
    ],
    'Inflatable Thin-Film Structures': [
        '- High modularity\n- Expandable arrays',
        '- Moderate efficiency\n- Thin-film limitations',
        '- Very lightweight\n- Thin-film materials',
        'Hundreds of meters',
        'A few tons',
        'MW–GW range',
        '$10–100 million'
    ],
    'Hypermodular SBSP Design': [
        '- Very high modularity\n- Emphasizes modular components',
        '- Variable efficiency\n- Depends on modules used',
        '- Lightweight\n- Small modules reduce mass',
        'Up to several km',
        '10,000–100,000 tons',
        'Up to several GW',
        '$50–500 billion'
    ],
    'Tethered Satellite System': [
        '- Moderate modularity\n- Connect multiple satellites via tethers',
        '- Can enhance efficiency\n- Distributes collection area',
        '- Lightweight\n- Tethers less massive than structures',
        'Several kilometers',
        '10,000–50,000 tons',
        '1–5 GW',
        '$100–500 million'
    ]
}

# Create the DataFrame
df = pd.DataFrame(data)

# Set 'Aspect' as the index
df.set_index('Aspect', inplace=True)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)

# Display the DataFrame
display(df)


,Large Array of Solar Panels,Sun Tower Design,Modular Symmetrical Concentrator,Solar Power Satellite via Arbitrarily Large Phased Array,Space-Based Reflectors,Inflatable Thin-Film Structures,Hypermodular SBSP Design,Tethered Satellite System
Aspect,,,,,,,,
Modularity,- Limited modularity\n- Monolithic structure,- Moderate modularity\n- Add collectors along tower,- High modularity\n- Interchangeable modules,- High modularity\n- Expandable phased array,- Moderate modularity\n- Add reflectors as needed,- High modularity\n- Expandable arrays,- Very high modularity\n- Emphasizes modular components,- Moderate modularity\n- Connect multiple satellites via tethers
Efficiency,- Good energy capture\n- Large surface area,- High efficiency\n- Constant solar exposure,- High efficiency\n- Concentrates sunlight,- High efficiency\n- Precise energy transmission,- Enhances efficiency\n- Redirects sunlight,- Moderate efficiency\n- Thin-film limitations,- Variable efficiency\n- Depends on modules used,- Can enhance efficiency\n- Distributes collection area
Lightweight,- Heavy\n- Large structural mass,- Heavy\n- Substantial structural components,- Moderate weight\n- Mirrors/lenses reduce mass,- Moderate to heavy\n- Large arrays increase mass,- Moderate weight\n- Lightweight materials possible,- Very lightweight\n- Thin-film materials,- Lightweight\n- Small modules reduce mass,- Lightweight\n- Tethers less massive than structures
Size,1–5 km,1–2 km height,1–2 km diameter,1–10 km,Meters to several km,Hundreds of meters,Up to several km,Several kilometers
Weight,"10,000–100,000 tons","5,000–20,000 tons","10,000–30,000 tons","10,000–100,000 tons",10–50 tons,A few tons,"10,000–100,000 tons","10,000–50,000 tons"
Energy Production,1–5 GW,0.5–1 GW,1–5 GW,1–10 GW,Boosts solar farms by +10%,MW–GW range,Up to several GW,1–5 GW
Cost,$20–500 billion,$10–100 billion,$20–150 billion,$20–500 billion,$0.5–1 billion,$10–100 million,$50–500 billion,$100–500 million


### References


Large array of solar panels:

- @article{mankins2009new,
  title={New directions for space solar power},
  author={Mankins, John C},
  journal={Acta Astronautica},
  volume={65},
  number={1-2},
  pages={146--156},
  year={2009},
  publisher={Elsevier}
}

- @inproceedings{rodgers2024space,
  title={Space Based Solar Power},
  author={Rodgers, Erica and Sotudeh, Jordan and Mullins, Carie and Hernandez, Amanda and Gertsen, Ellen and Joseph, Nikolai and Le, Hahn and Smith, Phil},
  booktitle={AIAA AVIATION FORUM AND ASCEND 2024},
  pages={4944},
  year={2024}
}


Sun Tower

- @article{mankins1997fresh,
  title={A fresh look at space solar power: New architectures, concepts and technologies},
  author={Mankins, John C},
  journal={Acta Astronautica},
  volume={41},
  number={4-10},
  pages={347--359},
  year={1997},
  publisher={Elsevier}
}


Modular Symmetrical Concentrator

- @book{mankins2014case,
  title={The Case for Space Solar Power},
  author={Mankins, J.},
  isbn={9780991337019},
  url={https://books.google.fr/books?id=WVyGoAEACAAJ},
  year={2014},
  publisher={Virginia Edition Publishing}
}


SPS via arbitarilly large phased array

- NASA Report RD1


Space based reflectors

- @book{national2007space,
  title={Space-based solar power as an opportunity for strategic security},
  author={National Space Society (US) and Rouge, Joseph D},
  year={2007},
  publisher={National Security Space Office}
}


Inflatable SBSP

- @inproceedings{litteken2019inflatable,
  title={Inflatable technology: using flexible materials to make large structures},
  author={Litteken, Douglas A},
  booktitle={Electroactive polymer actuators and devices (EAPAD) XXI},
  volume={10966},
  pages={1096603},
  year={2019},
  organization={SPIE}
}


Tether system 

- @article{tajmar2021bare,
  title={A bare-photovoltaic tether for consumable-less and autonomous space propulsion and power generation},
  author={Tajmar, Martin and S{\'a}nchez-Arriaga, Gonzalo},
  journal={Acta Astronautica},
  volume={180},
  pages={350--360},
  year={2021},
  publisher={Elsevier}
}